In [11]:
"""Access and manage a Google Tag Manager account."""

import argparse
import sys

import httplib2

from apiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools


def GetService(api_name, api_version, scope, client_secrets_path):
  """Get a service that communicates to a Google API.

  Args:
    api_name: string The name of the api to connect to.
    api_version: string The api version to connect to.
    scope: A list of strings representing the auth scopes to authorize for the
      connection.
    client_secrets_path: string A path to a valid client secrets file.

  Returns:
    A service that is connected to the specified API.
  """
  # Parse command-line arguments.
  parser = argparse.ArgumentParser(
      formatter_class=argparse.RawDescriptionHelpFormatter,
      parents=[tools.argparser])
  flags = parser.parse_args([])

  # Set up a Flow object to be used if we need to authenticate.
  flow = client.flow_from_clientsecrets(
      client_secrets_path, scope=scope,
      message=tools.message_if_missing(client_secrets_path))

  # Prepare credentials, and authorize HTTP object with them.
  # If the credentials don't exist or are invalid run through the native client
  # flow. The Storage object will ensure that if successful the good
  # credentials will get written back to a file.
  storage = file.Storage(api_name + '.dat')
  credentials = storage.get()
  if credentials is None or credentials.invalid:
    credentials = tools.run_flow(flow, storage, flags)
  http = credentials.authorize(http=httplib2.Http())

  # Build the service object.
  service = build(api_name, api_version, http=http)

  return service

def FindGreetingsContainer(service, account_path):
  """Find the greetings container.

  Args:
    service: the Tag Manager service object.
    account_path: the path of the Tag Manager account from which to retrieve the
      Greetings container.

  Returns:
    The greetings container if it exists, or None if it does not.
  """
  # Query the Tag Manager API to list all containers for the given account.
  container_wrapper = service.accounts().containers().list(
      parent=account_path).execute()

  # Find and return the Greetings container if it exists.
  for container in container_wrapper['container']:
    if container['name'] == 'Greetings':
      return container
  return None

def CreateWorkspace(service, container):
    """Creates a workspace named 'my workspace'.

    Args:
      service: the Tag Manager service object.
      container: the container to insert the workspace within.

    Returns:
      The created workspace.
    """
    return service.accounts().containers().workspaces().create(
        parent=container['path'],
        body={
            'name': 'my workspace',
        }).execute()

def CreateHelloWorldTag(service, workspace, tracking_id):
  """Create the Universal Analytics Hello World Tag.

  Args:
    service: the Tag Manager service object.
    workspace: the workspace to create a tag within.
    tracking_id: the Universal Analytics tracking ID to use.

  Returns:
    The created tag.
  """

  hello_world_tag = {
      'name': 'Universal Analytics Hello World',
      'type': 'ua',
      'parameter': [{
          'key': 'trackingId',
          'type': 'template',
          'value': str(tracking_id),
      }],
  }

  return service.accounts().containers().workspaces().tags().create(
      parent=workspace['path'],
      body=hello_world_tag).execute()


def CreateHelloWorldTrigger(service, workspace):
  """Create the Hello World Trigger.

  Args:
    service: the Tag Manager service object.
    workspace: the workspace to create the trigger within.

  Returns:
    The created trigger.
  """

  hello_world_trigger = {
      'name': 'Hello World Rule',
      'type': 'PAGEVIEW'
  }

  return service.accounts().containers().workspaces().triggers().create(
      parent=workspace['path'],
      body=hello_world_trigger).execute()

def UpdateHelloWorldTagWithTrigger(service, tag, trigger):
  """Update a Tag with a Trigger.

  Args:
    service: the Tag Manager service object.
    tag: the tag to associate with the trigger.
    trigger: the trigger to associate with the tag.
  """
  # Get the tag to update.
  tag = service.accounts().containers().workspaces().tags().get(
      path=tag['path']).execute()

  # Update the Firing Trigger for the Tag.
  tag['firingTriggerId'] = [trigger['triggerId']]

  # Update the Tag.
  response = service.accounts().containers().workspaces().tags().update(
      path=tag['path'],
      body=tag).execute()

# def main(argv):
#   # Get tag manager account ID from command line.
#   assert len(argv) == 2 and 'usage: gtm-api-hello-world.py <account_id>'
#   account_id = str(argv[1])

account_id = 6142867535
account_path = 'accounts/%s' % account_id

# Define the auth scopes to request.
scope = ['https://www.googleapis.com/auth/tagmanager.edit.containers']

# Authenticate and construct service.
service = GetService('tagmanager', 'v2', scope, '/Users/safiyymomen/Downloads/client_secret_758544568191-rssd6omutkskrjqtlbl9m4r1sb2kkih7.apps.googleusercontent.com.json')

# # Find the greetings container.
# container = FindGreetingsContainer(service, account_path)

# # Create a new workspace.
# workspace = CreateWorkspace(service, container)

# # Create the hello world tag.
# tag = CreateHelloWorldTag(
#   service, workspace, 'UA-1234-5')

# # Create the hello world Trigger.
# trigger = CreateHelloWorldTrigger(
#   service, workspace)

# # Update the hello world tag to fire based on the hello world tag.
# UpdateHelloWorldTagWithTrigger(service, tag, trigger)

# if __name__ == '__main__':
# main(sys.argv)


AssertionError: 

In [2]:
%pip install oauth2client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install --upgrade google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 1.2 MB/s eta 0:00:00m eta 0:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import json
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your JSON credentials file downloaded from the Google Cloud Console
credentials_path = '/Users/safiyymomen/Downloads/aero-396620-d26532d61477.json'

# Set up the credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/tagmanager.readonly']
)

# Create the Google Tag Manager API client
service = build('tagmanager', 'v2', credentials=credentials)

# Specify the account ID you want to retrieve data for
account_id = '6142867535'

# account = service.accounts().get(accountId=account_id).execute()


account_id = 6142867535
account_path = 'accounts/%s' % account_id



{'container': [{'path': 'accounts/6142867535/containers/139053661',
   'accountId': '6142867535',
   'containerId': '139053661',
   'name': 'getaero.io',
   'publicId': 'GTM-PMWN936N',
   'usageContext': ['web'],
   'fingerprint': '1692642700771',
   'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6142867535/containers/139053661/workspaces?apiLink=container',
   'features': {'supportUserPermissions': True,
    'supportEnvironments': True,
    'supportWorkspaces': True,
    'supportGtagConfigs': False,
    'supportBuiltInVariables': True,
    'supportClients': False,
    'supportFolders': True,
    'supportTags': True,
    'supportTemplates': True,
    'supportTriggers': True,
    'supportVariables': True,
    'supportVersions': True,
    'supportZones': True,
    'supportTransformations': False},
   'tagIds': ['GTM-PMWN936N']}]}

In [39]:
path

'accounts/6142867535/containers/139053661'

In [38]:
# containers = service.accounts().containers().list(parent=account_path).execute()
latest

{'path': 'accounts/6142867535/containers/139053661/versions/4',
 'accountId': '6142867535',
 'containerId': '139053661',
 'containerVersionId': '4',
 'name': 'Use native form submits',
 'numTags': '4',
 'numTriggers': '2'}

In [42]:
import json
for container in containers['container']:
    path = container['path']
    latest = service.accounts().containers().version_headers().latest(parent=path).execute()
    latest_pat = latest['path']
    print(json.dumps(latest, indent=4))

{
    "path": "accounts/6142867535/containers/139053661/versions/4",
    "accountId": "6142867535",
    "containerId": "139053661",
    "containerVersionId": "4",
    "name": "Use native form submits",
    "numTags": "4",
    "numTriggers": "2"
}


In [36]:
a = service.accounts().containers().versions()

In [37]:
a.get(path=account_path)

TypeError: Parameter "path" value "accounts/6142867535" does not match the pattern "^accounts/[^/]+/containers/[^/]+/versions/[^/]+$"